# Traitement des données pour un fichier
## Lecture d'un fichier csv

In [234]:
# imports des modules
import pandas as pd
import numpy as np
from scipy.signal import butter, lfilter
# faire les plot avec plotly plutot que matplotlib
pd.options.plotting.backend = "plotly"

Valeurs qu'il est possible de modifier pour changer le comportement du programme

In [235]:
PATH = '../donnees/export-chromato/AbaziM-20200210-CHROMATOGRAM.CSV'
PERIOD = 1000 # fréquence d'échantillonage en ms
RESAMPLE_MS = str(PERIOD) +'ms' # fréquence d'échantillonage en msS
INTERVAL = [6,45] # intervalle de temps observe
NOISE_FREQ = 0.02 # valeur à partir de laquelle sont éliminées les hautes fréquences en Hz
BIAS_FREQ = 0.001 # valeur de fréquence caractérisant le biais de la machine en Hz
spikes = [21.5,25.6, 30.2] # durée d'apparition des trois pics de référence (None si non présent)

In [236]:
# lecture du fichier csv
df = pd.read_csv(PATH, header=None, skiprows=[0,1,2,3,4,5], index_col=0)
#suppression de la colonne vide
df.drop(df.columns[1], axis=1, inplace=True)
df.rename(columns = {df.columns[0]: 'values'}, inplace=True)

On ne garde que les valeurs comprises dans l'interval qui nous interesse, cela supprime aussi les cases vide du csv (temps à 0)

In [237]:
df = df.drop(df[df.index > INTERVAL[1]].index)
df = df.drop(df[df.index < INTERVAL[0]].index)

Affichage des données non traitées brutes

In [238]:
df.plot()

Affichage des données en logarithme

In [239]:
df = np.log(df)
df.plot()

Normalisation des données : 

In [240]:
df = (df - df.mean())/df.std()
df.plot()

## Traitement pour éliminer le biais et avoir des intervalles de temps constant

Resampling de la courbe pour avoir un échantillonage constant <br>
On garde le maximum des valeurs lors au niveau de la fonction resample pour conserver les pics

In [241]:
df['time'] = pd.to_timedelta(df.index, 'min')  # conversion de la durée en timedelta pour pouvoir faire un resample
df.set_index('time', inplace=True) # met le temps en indice de la mesure
#print(df)
temp = df.resample(rule=RESAMPLE_MS).max() # on change l'échantillonage pour qu'il soit constant.
temp.plot()

Filtrage basse fréquence pour calculer le biais <br>
Soustraction de celui-ci aux données <br>
Tiré de : https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units

In [242]:
# création du filtre
def butter_lowpass(cutoff, ws, order=5):
    nyq = 0.5 * ws
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

# application du filtre
def butter_lowpass_filter(data, cutoff, fs, order=5):
    cutoff *= 2*np.pi # conversion en rad/s
    ws  = fs * 2 * np.pi
    print(cutoff, "    ", ws)
    b, a = butter_lowpass(cutoff, ws, order=order)
    y = lfilter(b, a, data)
    return y

BIAS_FREQ = 0.001

df['bias'] = butter_lowpass_filter(df['values'], BIAS_FREQ, 1/PERIOD*1000)
df.plot()

0.006283185307179587      6.283185307179586


Filtrage haute fréquence pour éliminer le bruit

In [243]:
df['corrected'] = butter_lowpass_filter(df['values'], NOISE_FREQ, 1/PERIOD*1000)
df['corrected'] = df['corrected'] - df['bias']
df.plot()

0.12566370614359174      6.283185307179586


In [244]:
df.loc[df['corrected'] < 0, 'corrected'] = 0
print(len(df['corrected']))
df['corrected'].plot()

6608
